## 0. Basic Analysis

#### 0.1 Pre-requisites

In [23]:
#Assign variables
trpv = 5
rep = 3

folder = f"../Simulations/TRPV{trpv}/replica_{rep}"

traj = f"{folder}/step7_production.xtc"
grof = f"{folder}/step7_production.gro"
gro0 = f"{folder}/CG_TRPV{trpv}.gro"
tpr = f"{folder}/step7_production.tpr"
energy = f"{folder}/step7_production.edr"

In [24]:
import os
#Create analysis directory
try:
    os.mkdir(f"{folder}/Analysis")
    print(f"Folder 'Analysis' created in {folder}")
except FileExistsError:
    print(f"Could not create folder. Folder 'Analysis' already exists in {folder}")

Folder 'Analysis' created in ../Simulations/TRPV5/replica_3


In [25]:
#Load required libraries
import MDAnalysis as mda
from MDAnalysis.analysis.leaflet import LeafletFinder
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from datetime import datetime
import subprocess
import re

### 0.2 System initial configuration

Analyze the system set up to check everything is correct.

This should include:
- Number of atoms (total and per molecule type)
- Number residues (total and per molecule type)
- Lipids per leaflet (absolute numbers, %, )
- System charge
- Contribution of protein, membrane, and ions to system charge

In [26]:
#Load .gro file of the initial coordinates
u0 = mda.Universe(tpr, gro0)

In [27]:
#Select the different molecule types
forcefield = subprocess.check_output(f"grep -i -o -m 1 'elnedyn22' {folder}/toppar/PROA_P.itp", shell=True, text=True)
pdb = re.search(r'\d[0-9a-z]{3}', subprocess.check_output(f"grep -i -o -m 1 '\-f .*_PROA.charmm.pdb' {folder}/toppar/PROA_P.itp", shell=True, text=True)).group(0)
system = u0.select_atoms("name *")
protein = u0.select_atoms("resname ALA ARG ASN ASP CYS GLN GLU GLY HIS ILE LEU LYS MET PHE PRO SER THR TRP TYR VAL")
protein_seq = u0.select_atoms(f'name BB and segid seg_0_PROA_P').resnames
lipids = {lipid:u0.select_atoms(f"resname {lipid}") for lipid in set(u0.select_atoms('name P* or resname CHOL').resnames)}
waters = u0.select_atoms("resname W")
NA = u0.select_atoms("name NA")
CL = u0.select_atoms("name CL")

#Select leaflets
L = LeafletFinder(u0, 'name PO4 ROH', cutoff=16)
upper_leaflet = L.groups(0)
lower_leaflet = L.groups(1)
upper_composition = {lipid:list(upper_leaflet.resnames).count(lipid) for lipid in set(upper_leaflet.resnames)}
lower_composition = {lipid:list(lower_leaflet.resnames).count(lipid) for lipid in set(lower_leaflet.resnames)}
upper_perc = {lipid:upper_composition[lipid]/sum(upper_composition.values())*100 for lipid in upper_composition.keys()}
lower_perc = {lipid:lower_composition[lipid]/sum(lower_composition.values())*100 for lipid in lower_composition.keys()}

In [28]:
with open(f"{folder}/Analysis/System_Data_t0.txt", "w") as f:
    f.write(f"SYSTEM SUMMARY FOR TRPV{trpv}, REPLICA {rep}\n")
    f.write(f"Created on {datetime.now().date()} {datetime.now().time()}\n\n\n")
    f.write(f"Initial box dimensions: X:{u0.dimensions[0]} A, Y:{u0.dimensions[1]} A, Z:{u0.dimensions[2]} A\n")
    f.write(f"Force Field: {forcefield}")
    f.write(f"PDB ID (from .itp file): {pdb}\n\n")
    f.write(f"Overall Composition:\n")
    f.write(f"\tSystem size: {len(system.names)} atoms, {len(set(system.resids))} residues\n")
    f.write(f"\tProtein size: {len(protein.names)} atoms, {len(set(protein.resids))} residues, {len(set(protein.segids))} chains\n")
    f.write(f"\tMembrane size: {sum([len(l.names) for l in lipids.values()])} atoms, {sum([len(set(l.resids)) for l in lipids.values()])} residues, {sum([len(set(l.resnames)) for l in lipids.values()])} lipid types\n")
    f.write(f"\tWaters: {len(waters.names)} atoms\n")
    f.write(f"\tNa+ ions: {len(NA.names)} atoms\n")
    f.write(f"\tCl- ions: {len(CL.names)} atoms\n")
    noat = (len(system.names) == sum([len(protein.names), sum([len(l.names) for l in lipids.values()]), len(waters.names), len(NA.names), len(CL.names)]))
    f.write(f"\tNumber of atoms match? {noat}\n\n")
    f.write(f"Charge Analysis:\n")
    f.write(f"\tSystem charge: {sum(system.charges)}\n")
    f.write(f"\tProtein charge: {sum(protein.charges)}\n")
    f.write(f"\tMembrane charge: {sum([sum(l.charges) for l in lipids.values()])}\n")
    f.write(f"\tWaters charge: {sum(waters.charges)}\n")
    f.write(f"\tNa+ ions charge: {sum(NA.charges)}\n")
    f.write(f"\tCl- ions charge: {sum(CL.charges)}\n")
    f.write(f"\tNet Protein + Membrane Charge: {sum([sum(l.charges) for l in lipids.values()])+sum(protein.charges)}\n")
    f.write(f"\tNet Ion Charge: {sum(NA.charges)+sum(CL.charges)}\n\n")
    f.write(f"Membrane Composition Analysis:\n")
    f.write(f"\tAbsolute Number of Lipids:\n")
    f.write(f"\t\t\t\tUPPER\t\tLOWER\n")
    for l in lipids.keys():
        up = upper_composition.get(l, "-")
        lo = lower_composition.get(l, "-")
        f.write(f"\t\t{l}\t{up}\t\t\t{lo}\n")
    f.write(f"\t\tTOTAL\t{sum([upper_composition.get(l,0) for l in lipids.keys()])}\t\t{sum([lower_composition.get(l,0) for l in lipids.keys()])}\n\n")
    f.write(f"\tRelative Composition:\n")
    f.write(f"\t\t\t\tUPPER\t\tLOWER\n")
    for l in lipids.keys():
        up = round(upper_perc.get(l, 0.0),1)
        lo = round(lower_perc.get(l, 0.0), 1)
        line = f"\t\t{l}\t{up}%\t\t{lo}%\n"
        f.write(line.replace('0.0%', ' - %'))
    f.write(f"\t\tTOTAL\t{sum([upper_perc.get(l,0) for l in lipids.keys()])}\t\t{sum([lower_perc.get(l,0) for l in lipids.keys()])}\n")
    
    f.write(f"\nProtein sequence:\n {' '.join(protein_seq)}\n\n")




### 0.3 Extract and plot simulation data
- Temperature
- Pressure
- Volume
- Density
- Energies
- Mindists to PB images
- Protein backbone RMSD


In [29]:
#Create stats folder
try:
    os.mkdir(f"{folder}/Analysis/Stats")
    print("Folder Analysis/Stats created")
except OSError as error:
    print("Can't create folder: folder Analysis/Stats already exists")

Folder Analysis/Stats created


In [30]:
#Select data from .edr to extract
variables = ["temperature", "density", "pressure", "volume", "potential", "kinetic-en", "total-energy"]

for variable in variables:
    command = f"echo {variable}|gmx energy -f {energy} -o {folder}/Analysis/Stats/{variable}.xvg"
    subprocess.run(["/bin/bash", "-i", "-c", f"source ~/.bashrc && activate_gmx2020 && {command}"])

bash: no se puede establecer el grupo de proceso de terminal (133709): Función ioctl no apropiada para el dispositivo
bash: no hay control de trabajos en este intérprete de ordenes
                      :-) GROMACS - gmx energy, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons S


Statistics over 250000001 steps [ 0.0000 through 5000000.0000 ps ], 1 data sets
All statistics are over 2500001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 309.609    0.00097   0.857594 -0.00545889  (K)

Statistics over 250000001 steps [ 0.0000 through 5000000.0000 ps ], 1 data sets
All statistics are over 2500001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Density                      1034.6     0.0035   0.745229  0.0193684  (kg/m^3)


                      :-) GROMACS - gmx energy, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     


Statistics over 250000001 steps [ 0.0000 through 5000000.0000 ps ], 1 data sets
All statistics are over 2500001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                    1.74012     0.0051    15.4224  0.0159831  (bar)

Statistics over 250000001 steps [ 0.0000 through 5000000.0000 ps ], 1 data sets
All statistics are over 2500001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Volume                      20726.3      0.071    14.9294  -0.388063  (nm^3)


                      :-) GROMACS - gmx energy, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     


Statistics over 250000001 steps [ 0.0000 through 5000000.0000 ps ], 1 data sets
All statistics are over 2500001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                -4.63916e+06         97    2816.37   -665.707  (kJ/mol)

Statistics over 250000001 steps [ 0.0000 through 5000000.0000 ps ], 1 data sets
All statistics are over 2500001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Kinetic En.                  683654        2.2    1893.67   -12.0518  (kJ/mol)

Statistics over 250000001 steps [ 0.0000 through 5000000.0000 ps ], 1 data sets
All statistics are over 2500001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Total Energy             -3.95

                      :-) GROMACS - gmx energy, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

In [31]:
#Function to read .xvg files into a proper DataFrame to plot later, removing the header, first column is time, second is the variable
def read_xvg(file, x_name, y_name, skip_lines):
    data = {x_name: [], y_name:[]} #Empty dict to store the data
    temp_file = open(file, 'r')
    temp_lines = temp_file.readlines()[skip_lines:] #This removes header lines

    for line in temp_lines: #Iterate, split data by space, get first number (time) and second (variable) and store in dictionary
        values = line.split()
        data[x_name].append(float(values[0])); data[y_name].append(float(values[1]))

    df = pd.DataFrame(data)
    return df

#Function to get average of variable in a window-sliding fashion
def get_avg(df, column_name, window):
    df['Avg'] = df[column_name]
    for i in range(len(df[column_name])):
        if i < window:
            df['Avg'][i] = None #Ignore rows until window size
        else:
            df['Avg'][i] = np.mean(df[column_name][(i-window):i]) #Add mean column
    return df

In [32]:
#Terms to plot for quality analysis
dict_terms = {'temperature': 'Temperature (K)', 'pressure': 'Pressure (bars)', 
              'volume': 'Volume ($nm3$)', 'density': 'Density (kg/$nm^3$)', 
              'potential': 'Energy (kJ/mol)', 'kinetic-en': 'Energy (kJ/mol)'}

for k,v in dict_terms.items():
    print(f"Plotting {k}")
    term_df = read_xvg(f"{folder}/Analysis/Stats/{k}.xvg", 'Time', k, 24) #Read file .xvg into a dataframe

    term_df = get_avg(term_df, k, 100) #Get avgs every 50 frames
    
    term_df["Time"] = term_df["Time"]/1000000

    #Plot
    plt.style.use('seaborn-colorblind')  ####### Style for the plots (defined one to keep same style accross ALL plots ########
    sns_term = sns.lineplot(x='Time', y=k, data=term_df) #Plot term over time
    sns_temp = sns.lineplot(x='Time', y='Avg', data=term_df) #Plot avg term over time
    sns_temp.set_xlabel('Time (μs)')
    sns.despine()
    y_ax_label = v #get y axis name based on term being plotted
    sns_temp.set_ylabel(y_ax_label) #Change y axis label
    fig = sns_temp.get_figure()  # Get figure
    output_name = f"{folder}/Analysis/Stats/{k}.svg"
    fig.savefig(output_name,dpi=300, bbox_inches='tight')  # Save graph in folder
    plt.clf()  # Erase plot to avoid stacking one on each other

Plotting temperature


/tmp/ipykernel_135830/3559799261.py:15: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-colorblind')  ####### Style for the plots (defined one to keep same style accross ALL plots ########


Plotting pressure


/tmp/ipykernel_135830/3559799261.py:15: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-colorblind')  ####### Style for the plots (defined one to keep same style accross ALL plots ########


Plotting volume


...

Plotting potential


/tmp/ipykernel_135830/3559799261.py:15: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-colorblind')  ####### Style for the plots (defined one to keep same style accross ALL plots ########


Plotting kinetic-en


/tmp/ipykernel_135830/3559799261.py:15: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-colorblind')  ####### Style for the plots (defined one to keep same style accross ALL plots ########


<Figure size 640x480 with 0 Axes>

### 0.4 Center trajectory

Take water and ions out to minimize time 

In [33]:
new_index = f"echo -e '1|40\nq' | gmx make_ndx -f {grof} -n {folder}/index.ndx -o {folder}/Analysis/analysis_index.ndx"
subprocess.run(["/bin/bash", "-i", "-c", f"source ~/.bashrc && activate_gmx2020 && {new_index}"])

new_coord = f"echo 42 | gmx trjconv -f {traj} -s {tpr} -n {folder}/Analysis/analysis_index.ndx -pbc nojump -dump -1 -o {folder}/Analysis/analysis_step7_coord.gro"
subprocess.run(["/bin/bash", "-i", "-c", f"source ~/.bashrc && activate_gmx2020 && {new_coord}"])

step1 = f'echo System | gmx trjconv -f {traj} -s {tpr} -n {folder}/Analysis/analysis_index.ndx -pbc nojump -o {folder}/Analysis/tra1.xtc'
subprocess.run(["/bin/bash", "-i", "-c", f"source ~/.bashrc && activate_gmx2020 && {step1}"])

step2 = f'echo Protein System | gmx trjconv -f {folder}/Analysis/tra1.xtc -s {tpr} -n {folder}/Analysis/analysis_index.ndx -fit translation -o {folder}/Analysis/tra2.xtc'
subprocess.run(["/bin/bash", "-i", "-c", f"source ~/.bashrc && activate_gmx2020 && {step2}"])
os.system(f'rm {folder}/Analysis/tra1.xtc')

final_step = f'echo 42 | gmx trjconv -f {folder}/Analysis/tra2 -s {tpr} -n {folder}/Analysis/analysis_index.ndx -pbc mol -o {folder}/Analysis/analysis_centered_step7_production.xtc'
subprocess.run(["/bin/bash", "-i", "-c", f"source ~/.bashrc && activate_gmx2020 && {final_step}"])
os.system(f'rm {folder}/Analysis/tra2.xtc')


bash: no se puede establecer el grupo de proceso de terminal (133709): Función ioctl no apropiada para el dispositivo
bash: no hay control de trabajos en este intérprete de ordenes
                     :-) GROMACS - gmx make_ndx, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons 

Going to read 1 old index file(s)

  0 System              : 182151 atoms
  1 protein             :  5464 atoms
  2 Protein-H           :  5464 atoms
  3 C-alpha             :     0 atoms
  4 Backbone            :     0 atoms
  5 MainChain           :     0 atoms
  6 MainChain+Cb        :     0 atoms
  7 MainChain+H         :     0 atoms
  8 SideChain           :  5464 atoms
  9 SideChain-H         :  5464 atoms
 10 Prot-Masses         :  5464 atoms
 11 non-Protein         : 176687 atoms
 12 Other               : 173144 atoms
 13 CHOL                :  7368 atoms
 14 POPC                :  6768 atoms
 15 PAPC                :  3653 atoms
 16 POPE                :  1884 atoms
 17 DIPE                :  3180 atoms
 18 DPSM                :  5841 atoms
 19 PAPS                :  3055 atoms
 20 PAP6                :   272 atoms
 21 POPA                :   176 atoms
 22 PAP1                :   256 atoms
 23 W                   : 140691 atoms
 24 NA+                 :  1989 atoms
 25 CL-    

Group     0 (         System) has 182151 elements
Group     1 (        protein) has  5464 elements
Group     2 (      Protein-H) has  5464 elements
Group     3 (        C-alpha) has     0 elements
Group     4 (       Backbone) has     0 elements
Group     5 (      MainChain) has     0 elements
Group     6 (   MainChain+Cb) has     0 elements
Group     7 (    MainChain+H) has     0 elements
Group     8 (      SideChain) has  5464 elements
Group     9 (    SideChain-H) has  5464 elements
Group    10 (    Prot-Masses) has  5464 elements
Group    11 (    non-Protein) has 176687 elements
Group    12 (          Other) has 173144 elements
Group    13 (           CHOL) has  7368 elements
Group    14 (           POPC) has  6768 elements
Group    15 (           PAPC) has  3653 elements
Group    16 (           POPE) has  1884 elements
Group    17 (           DIPE) has  3180 elements
Group    18 (           DPSM) has  5841 elements
Group    19 (           PAPS) has  3055 elements
Group    20 (    

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 42: 'protein_membrane'


Group     0 (         System) has 182151 elements
Group     1 (        protein) has  5464 elements
Group     2 (      Protein-H) has  5464 elements
Group     3 (        C-alpha) has     0 elements
Group     4 (       Backbone) has     0 elements
Group     5 (      MainChain) has     0 elements
Group     6 (   MainChain+Cb) has     0 elements
Group     7 (    MainChain+H) has     0 elements
Group     8 (      SideChain) has  5464 elements
Group     9 (    SideChain-H) has  5464 elements
Group    10 (    Prot-Masses) has  5464 elements
Group    11 (    non-Protein) has 176687 elements
Group    12 (          Other) has 173144 elements
Group    13 (           CHOL) has  7368 elements
Group    14 (           POPC) has  6768 elements
Group    15 (           PAPC) has  3653 elements
Group    16 (           POPE) has  1884 elements
Group    17 (           DIPE) has  3180 elements
Group    18 (           DPSM) has  5841 elements
Group    19 (           PAPS) has  3055 elements
Group    20 (    

...

Group     0 (         System) has 182151 elements
Group     1 (        protein) has  5464 elements
Group     2 (      Protein-H) has  5464 elements
Group     3 (        C-alpha) has     0 elements
Group     4 (       Backbone) has     0 elements
Group     5 (      MainChain) has     0 elements
Group     6 (   MainChain+Cb) has     0 elements
Group     7 (    MainChain+H) has     0 elements
Group     8 (      SideChain) has  5464 elements
Group     9 (    SideChain-H) has  5464 elements
Group    10 (    Prot-Masses) has  5464 elements
Group    11 (    non-Protein) has 176687 elements
Group    12 (          Other) has 173144 elements
Group    13 (           CHOL) has  7368 elements
Group    14 (           POPC) has  6768 elements
Group    15 (           PAPC) has  3653 elements
Group    16 (           POPE) has  1884 elements
Group    17 (           DIPE) has  3180 elements
Group    18 (           DPSM) has  5841 elements
Group    19 (           PAPS) has  3055 elements
Group    20 (    

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for translational fit
Selected 1: 'protein'
Select group for output
Selected 0: 'System'


bash: no se puede establecer el grupo de proceso de terminal (133709): Función ioctl no apropiada para el dispositivo
bash: no hay control de trabajos en este intérprete de ordenes
                     :-) GROMACS - gmx trjconv, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons S

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 42: 'protein_membrane'


0